In [1]:
!nvidia-smi

Wed Jul 13 10:03:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:1A:00.0 Off |                  Off |
| 34%   28C    P8    32W / 260W |   9219MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:1C:00.0 Off |                  Off |
| 33%   

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.filterwarnings("ignore")

from Parse_TFrecords import *
from define_model import *
from load_label import *
from utilities import *
import gc
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import numpy as np
from sklearn.metrics import classification_report, average_precision_score

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Model Ensembling 

In [3]:
BATCH_SIZE = 32

pos = 72917
neg = 263952

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

## Prediction-based ensemble

In [4]:
BATCH_SIZE = 32
record_file_test = 'tfrecords/copd_mimic_test.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_test = get_data_label('mimic', 'test')

In [ ]:
archis = ['DenseNet121', 'ResNet50V2', 'MobileNetV2', 'InceptionResNetV2', 'Xception',  'InceptionV3']

preds = []
for archi in archis:
    checkpoint_filepath = 'checkpoints_mimic/checkpoint_BCE_{i}'.format(i=archi)
    model = define_model(archi)
    model.load_weights(checkpoint_filepath)

    preds.append(model.predict(test_dataset))

preds = np.array(preds)

In [ ]:
y_preds_ave = 0
for i in range(6):
    y_preds_ave += preds[i]

y_preds_ave = y_preds_ave/6
        
test_CI(y_preds_ave, y_test)

print(average_precision_score(y_test, y_preds_ave, average=None))
    
thresh = get_thresh(y_test, y_preds_ave, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_ave >= thresh, 1, 0)))

gc.collect()

In [ ]:
y_preds_wei = 0
for i in range(6):
    if (i < 3):
        y_preds_wei += preds[i]*3
    elif (i < 4):
        y_preds_wei += preds[i]*2
    else:
        y_preds_wei += preds[i]

y_preds_wei = y_preds_wei/13
        
test_CI(y_preds_wei, y_test)

print(average_precision_score(y_test, y_preds_wei, average=None))
    
thresh = get_thresh(y_test, y_preds_wei, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_wei >= thresh, 1, 0)))

gc.collect()